# Fix gene ids in iYali map

This notebook fixes the iYali gene ids available at https://caffeine.dd-decaf.eu/

Benjamín J. Sánchez, 2020-03-23

## 1. Getting model and map

* Model: https://api.dd-decaf.eu/model-storage/models/44
* Map: https://api.dd-decaf.eu/map-storage/maps/37

In [1]:
import wget
import os

def deleteIfExists(filename):
    if os.path.exists(filename):
        os.remove(filename)

deleteIfExists("iYali-model.json")
deleteIfExists("iYali-map.json")
wget.download("https://api.dd-decaf.eu/model-storage/models/44", "iYali-model.json", bar=False)
wget.download("https://api.dd-decaf.eu/map-storage/maps/37", "iYali-map.json", bar=False)

'iYali-map.json'

Let's check if they are the same in staging:

* Model: https://api-staging.dd-decaf.eu/model-storage/models/44
* Map: https://api-staging.dd-decaf.eu/map-storage/maps/90

In [2]:
deleteIfExists("iYali-model.json")
deleteIfExists("iYali-map.json")
wget.download("https://api-staging.dd-decaf.eu/model-storage/models/44", "iYali-model.json", bar=False)
wget.download("https://api-staging.dd-decaf.eu/map-storage/maps/90", "iYali-map.json", bar=False)  # note the id is different

'iYali-map.json'

So the model is the same (no changes in git) but the map is different. To what is exactly different, we first need to store the json files in a readable format (currently they are stored in one line):

In [3]:
# Go back to the main version:
deleteIfExists("iYali-model.json")
deleteIfExists("iYali-map.json")
wget.download("https://api.dd-decaf.eu/model-storage/models/44", "iYali-model.json", bar=False)
wget.download("https://api.dd-decaf.eu/map-storage/maps/37", "iYali-map.json", bar=False)

# Load object:
import json
def openJSON(filename):
    with open(filename, 'r') as handle:
        parsed = json.load(handle)
        handle.close()
    return parsed

# Convert to readable format:
def storePrettyJSON(parsed, filename):
    # Generate pretty text from stored file:
    pretty_text = json.dumps(parsed, indent=4, sort_keys=True)
    # Store pretty text as file:
    with open(filename, "w") as handle:
        handle.write(pretty_text)
        handle.close()

# Function for doing both:
def makeJSONpretty(filename):
    parsed = openJSON(filename)
    storePrettyJSON(parsed, filename)

makeJSONpretty("iYali-model.json")
makeJSONpretty("iYali-map.json")

Let's repeat the analysis to see what has changed in staging:

In [4]:
deleteIfExists("iYali-model.json")
deleteIfExists("iYali-map.json")
wget.download("https://api-staging.dd-decaf.eu/model-storage/models/44", "iYali-model.json", bar=False)
wget.download("https://api-staging.dd-decaf.eu/map-storage/maps/90", "iYali-map.json", bar=False)
makeJSONpretty("iYali-model.json")
makeJSONpretty("iYali-map.json")

So actually the only thing that changed was the map id (see git), the map itself is the same. Great! We can proceed by only fixing one model and one map. Let's get the relevant parts out of the caffeine-specific objects and save them again as pretty JSON:

In [5]:
# Load models
iYali_model = openJSON("iYali-model.json")
iYali_map = openJSON("iYali-map.json")

# Get relevant parts:
iYali_model = iYali_model["model_serialized"]
iYali_map = iYali_map["map"]

# Store the new elements:
storePrettyJSON(iYali_model, "iYali-model.json")
storePrettyJSON(iYali_map, "iYali-map.json")

And in the case of the model, let's load it / save it as a model using cobrapy:

In [6]:
import cobra
iYali_model = cobra.io.load_json_model("iYali-model.json")
iYali_model

Using license file C:\Users\bejsab\gurobi.lic
Academic license - for non-commercial use only


Name,iYali
Memory address,0x02577a6408c8
Number of metabolites,1671
Number of reactions,1924
Number of groups,0
Objective expression,1.0*xBIOMASS - 1.0*xBIOMASS_reverse_52bad
Compartments,"cell envelope, extracellular, mitochondrion, cytoplasm, peroxisome, endoplasmic reticulum, nucleus, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, mitochondrial membrane, Golgi membrane"


In [7]:
cobra.io.save_json_model(iYali_model, "iYali-model.json")

## 2. Fix model

First we will fix the model. The only thing needed is to swap id for name in each gene, cobrapy takes care of the rest:

In [8]:
gene_dict = {}
for gene in iYali_model.genes:
    # Store dictionary (old, new):
    gene_dict[gene.id] = gene.name
    # Change gene name:
    gene.name = gene.id

# Apply dictionary to change all ids using cobra function:
cobra.manipulation.modify.rename_genes(iYali_model, gene_dict)

# Save new model:
cobra.io.save_json_model(iYali_model, "iYali-model.json")